In [ ]:
import subprocess
import re
import tkinter as tk
from tkinter import *
from tkinter.ttk import *


window = tk.Tk()
window.title("Wi-Fi Connection Manager")
window.geometry('900x600') 
window.resizable(width=False, height=False)
window.config(bg='white')


def read_data_from_cmd():
    ssid2 = []
    res = {}
    p = subprocess.Popen("netsh wlan show networks mode=BSSID", stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out = p.stdout.read().decode('unicode_escape').strip()

    
    ssid = re.findall(r'SSID.*:.*?([A-z0-9 \-]*)', out)
    sign = re.findall(r'Signal.*?:.*?([0-9]*)%', out)
    
    for i in range(0, len(ssid), 2):
        ssid2.append(ssid[i])
    
    for i in range(len(ssid2)):
        res.update({ssid2[i]: sign[i]})
    
    
    res = sorted(res.items(), key=lambda item: int(item[1]), reverse=True)

    p.communicate()
    return res


def update():
    global res
    res = read_data_from_cmd()
    b = {}  
    k = {}  

   
    for widget in window.winfo_children():
        widget.destroy()

    
    for i in range(len(res)):
        Label(window, background="white", text=res[i][0] + '    ', font=("arial", 15)).grid(row=i+1, column=0, padx=10, pady=5)
        Label(window, background="white", text="Puissance: " + res[i][1] + "%", font=("arial", 15)).grid(row=i+1, column=1, padx=10, pady=5)

        # Connect button
        def connect_func(x=res[i][0]):
            return connect(x)
        b[i] = tk.Button(window, background="green", text="Connect", command=connect_func, font=("arial", 12, 'bold'), fg="white")
        b[i].grid(row=i+1, column=2, padx=10, pady=5)

        # Disconnect button
        def disconnect_func(x=res[i][0]):
            return disconnect(x)
        k[i] = tk.Button(window, background="red", text="Disconnect", command=disconnect_func, font=("arial", 12, 'bold'), fg="white")
        k[i].grid(row=i+1, column=3, padx=10, pady=5)

    # Button to connect to the best-performing network
    tk.Button(window, text="Connect to Best", font=("Courier", 12, 'bold'), background="blue", foreground='white', command=connect_best).grid(row=0, column=2, padx=10, pady=10)

    window.after(6000, update)  


def connect(ssid):
    p = subprocess.Popen(f'netsh wlan connect name="{ssid.strip()}"', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    conn_out = p.stdout.read().decode('unicode_escape').strip()
    print(f"Connecting to: {ssid.strip()}")
    print(conn_out)


def disconnect(ssid):
    p = subprocess.Popen(f'netsh wlan disconnect', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    disconn_out = p.stdout.read().decode('unicode_escape').strip()
    print(f"Disconnecting from: {ssid.strip()}")
    print(disconn_out)


def connect_best():
    res = read_data_from_cmd()
    connect(res[0][0])


update()


window.mainloop()


Connecting to: Ooredoo 4G_8A7073
Connection request was completed successfully.
Disconnecting from: Ooredoo 4G_8A7073
Disconnection request was completed successfully for interface "Wi-Fi".
Connecting to: Ooredoo 4G_8A7073
Connection request was completed successfully.
